In [1]:
import torch
from torch import nn

## 参数访问

In [2]:
net = nn.Sequential(
    nn.Linear(4, 8), 
    nn.ReLU(),
    nn.Linear(8, 1)
)
x = torch.randn((2, 4))
net

tensor([[-0.1326],
        [-0.3060]], grad_fn=<AddmmBackward0>)

In [9]:
## 目标参数
net[2].state_dict(), type(net[2].bias), net[2].bias.data, net[2].weight.shape

(OrderedDict([('weight',
               tensor([[ 0.3465, -0.1745,  0.2809, -0.2745,  0.2195,  0.1831, -0.2188,  0.3463]])),
              ('bias', tensor([-0.2913]))]),
 torch.nn.parameter.Parameter,
 tensor([-0.2913]),
 torch.Size([1, 8]))

In [10]:
net[2].weight.grad is None

True

In [11]:
## 一次性访问所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [12]:
net.state_dict()['2.bias'].data

tensor([-0.2913])

In [13]:
## 从嵌套块收集参数
def block1():
    return nn.Sequential(
        nn.Linear(4, 8),
        nn.ReLU(),
        nn.Linear(8, 4),
        nn.ReLU()
    )
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(x)

tensor([[0.5062],
        [0.5064]], grad_fn=<AddmmBackward0>)

In [14]:
rgnet

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

In [15]:
rgnet[0][1][0].bias.data

tensor([-0.2555,  0.3494,  0.2137,  0.1560, -0.3964,  0.2218, -0.0094, -0.4315])

## 参数初始化

In [21]:
## 内置初始化
def init_normal(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, 0, 0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0156,  0.0049,  0.0010,  0.0028]), tensor(0.))

In [31]:
def init_constant(m):
    if isinstance(m, nn.Linear):
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [32]:
def init_xavier(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
def init_42(m):
    if isinstance(m, nn.Linear):
        nn.init.constant_(m.weight, 42)
net[0].apply(init_xavier)
net[2].apply(init_42)
net[0].weight.data[0], net[2].weight.data

(tensor([-0.5587, -0.2119, -0.5813,  0.4862]),
 tensor([[42., 42., 42., 42., 42., 42., 42., 42.]]))

In [37]:
## 自定义初始化
def my_init(m):
    if isinstance(m, nn.Linear):
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000, -9.0324, -0.0000,  0.0000],
        [-0.0000,  0.0000,  0.0000,  0.0000]], grad_fn=<SliceBackward0>)

## 参数绑定

In [38]:
shared = nn.Linear(8, 8)
net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    shared,
    nn.ReLU(),
    shared,
    nn.ReLU(),
    nn.Linear(8, 1)
)
net(x)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])
net

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=8, bias=True)
  (5): ReLU()
  (6): Linear(in_features=8, out_features=1, bias=True)
)

## Exercises

1. 使用 5.1节 中定义的FancyMLP模型，访问各个层的参数

2. 查看初始化模块文档以了解不同的初始化方法

3. 构建包含共享参数层的多层感知机并对其进行训练。在训练过程中，观察模型各层的参数和梯度

4. 为什么共享参数是个好主意

共享参数通常可以节省内存，并在以下方面具有特定的好处：

- 对于图像识别中的CNN，共享参数使网络能够在图像中的任何地方而不是仅在某个区域中查找给定的功能
- 对于RNN，它在序列的各个时间步之间共享参数，因此可以很好地推广到不同序列长度的示例
- 对于自动编码器，编码器和解码器共享参数。 在具有线性激活的单层自动编码器中，共享权重会在权重矩阵的不同隐藏层之间强制正交